In [1]:
! ls

build-train-test-deploy-endpoint-in-sagemaker.ipynb  user-default-efs
mob_price_classification_train.csv


In [1]:
! pwd

/home/sagemaker-user


In [2]:
! echo $USER

sagemaker-user


In [3]:
! date

Tue Nov  5 12:04:54 UTC 2024


## _`Tutorial - 1 Mobile Price Classification using SKLearn Custom Script in Sagemaker`_

### _`Let's divide the workload`_
1. Initialize Boto3 SDK and create S3 bucket. 
2. Upload data in Sagemaker Local Storage. 
3. Data Exploration and Understanding.
4. Split the data into Train/Test CSV File. 
5. Upload data into the S3 Bucket.
6. Create Training Script
7. Train script in-side Sagemaker container. 
8. Store Model Artifacts(model.tar.gz) into the S3 Bucket. 
9. Deploy Sagemaker Endpoint(API) for trained model, and test it. 

In [5]:
import sklearn 

In [6]:
sklearn.__version__

'1.4.2'

_`We are not going to perform a Sklearn algorithm training into our local environment (local PC), we will perform the training into the sagemaker container. But, first, we are going to create a training script. So, we are going to use sklearn custom script, and there, we will be giving it to sagemaker and so that sagemaker can do the training and create an endpoint for us.`_

## _`1. Initialize Boto3 SDK and create S3 bucket. `_

In [7]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame as DF, Series as Srx 

import sagemaker
from sagemaker import get_execution_role # sagemaker needs permissions (roles) to be able to access other AWS services (E.g: S3) for training and testing channels

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

import datetime
import time 

import tarfile
import boto3

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [8]:
# We need a boto3 sagemaker client if we want to communicate with sagemaker from the notebook
boto3_sagemaker_client = boto3.client('sagemaker') 

In [9]:
# We will start a sagemaker session in order to be able to get the region name without hardcoding it. 
# It is a best practice in production environment
sagemaker_session = sagemaker.Session() 

In [10]:
region1: str = sagemaker_session.boto_session.region_name

In [11]:
region1

'eu-west-3'

In [12]:
region: str = sagemaker_session.boto_region_name

In [13]:
region

'eu-west-3'

In [14]:
bucket_name: str = 'sagemaker-diangana-bucket'  # This bucket should have been already created 

print(f'We are using the bucket\t: {bucket_name}')

We are using the bucket	: sagemaker-diangana-bucket


## _`We could choose the default bucket of my current sagemaker domain if we wanted`_

In [15]:
sagemaker_session.default_bucket()

'sagemaker-eu-west-3-050752617294'

## _`2. Upload data (CSV file) in Sagemaker Local Storage. `_

_`Done`_

## _`3. Data Exploration and Understanding.`_

In [16]:
df: DF = pd.read_csv(filepath_or_buffer='mob_price_classification_train.csv')

In [17]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [18]:
df.shape

(2000, 21)

In [19]:
df.isna().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [20]:
# ['Low_Risk','High_Risk'],[0,1]
df['price_range'].value_counts(normalize=True)

price_range
1    0.25
2    0.25
3    0.25
0    0.25
Name: proportion, dtype: float64

In [21]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [22]:
# Find the Percentage of Values are missing
df.isnull().mean() * 100

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [23]:
# features = list(df.columns)
# features

In [24]:
# label = features.pop(-1)
# label

In [25]:
# X = df[features]
# y = df[label]

In [26]:
X, y = df.drop('price_range', axis=1), df['price_range']

In [27]:
X.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [28]:
# {0: 'Low_Risk',1: 'High_Risk'}
y.head()

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [29]:
X.shape

(2000, 20)

In [30]:
y.value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

### _`Train Test Split`_

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)

In [32]:
print(f'X_train shape is: \t {X_train.shape}')
print(f'y_train shape is: \t {y_train.shape}')

print('\n\n')
print(f'X_test shape is: \t {X_test.shape}')
print(f'y_test shape is: \t {y_test.shape}')


X_train shape is: 	 (1700, 20)
y_train shape is: 	 (1700,)



X_test shape is: 	 (300, 20)
y_test shape is: 	 (300,)


## _`4. Split the data into Train/Test CSV File.`_

_`Let's split the data into Train/Test CSV Files and upload them into the 'sagemaker-diangana-bucket' S3 bucket. Beacuse sagemaker container required that both the training and testing datasets being present in S3 bucket`_

In [33]:
train_data_df: DF = pd.DataFrame(X_train) # Let's create a dataframe of the X_train so that we can save it as a csv file and then upload it to S3
train_data_df['price_range'] = y_train # Let's add the y colum to the training set

test_data_df: DF = pd.DataFrame(X_test) # Let's create a dataframe of the X_test so that we can save it as a csv file and then upload it to S3
test_data_df['price_range'] = y_test # Let's add the y colum to the testing set

In [34]:
train_data_df.isna().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [35]:
test_data_df.isna().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

## _`5. Upload data into the S3 Bucket.`_

_`Let's first of all, save the two above training and testing pandas dataframes as CSV file into the sagemaker local storage`_

In [36]:
train_data_df.to_csv(path_or_buf='train.csv', index=False)
test_data_df.to_csv(path_or_buf='test.csv', index=False)

_`Let's now send both the 'train-V-1.csv' and 'test-V-1.csv' data to S3 since SageMaker will take and expect the training data from s3`_

In [39]:
# send data to S3. SageMaker will take training data from s3
training_data_prefix: str = "data/train"


training_data_path: str = sagemaker_session.upload_data(
    path="train.csv", bucket=bucket_name, key_prefix=training_data_prefix
)

In [40]:
# send data to S3. SageMaker will take training data from s3
testing_data_prefix: str = "data/test"

testing_data_path: str = sagemaker_session.upload_data(
    path="test.csv", bucket=bucket_name, key_prefix=testing_data_prefix
)

In [41]:
training_data_path

's3://sagemaker-diangana-bucket/data/train/train.csv'

In [42]:
testing_data_path

's3://sagemaker-diangana-bucket/data/test/test.csv'

## _`6. Create Training Script`_

_`Let's now create a training script (.py) that we will be giving to the sagemaker sklearn container. For that, we will be using the '%%writefile'magic cell command to write the bellow python script into a 'train.py' file `_ <br>

_`-a: means append to the content to the existing file, and -w (default) will overwrite the content or create the file if it doesn't exist yet `_



In [62]:
%%writefile train.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, \
                            recall_score, f1_score, roc_curve, auc
import sklearn
from sklearn.model_selection import GridSearchCV

import joblib
import boto3

import pathlib
from io import StringIO

import argparse
import os

import numpy as np 
import pandas as pd 
from pandas import DataFrame as DF, Series as Srx 


if __name__ == "__main__":
    print('[INFO] Extracting the passed arguments from the command line\n\n')

    parser = argparse.ArgumentParser()
    
    # Hyperparameters sent by the client are passed as command-line arguments to the training script script.
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # Let's now add parameters for Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--training-data-path", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--testing-data-path", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--training-file", type=str, default="train.csv")
    parser.add_argument("--testing-file", type=str, default="test.csv")
    # parser.add_argument("--output-data-path", type=str, default=os.environ.get("SM_OUTPUT_DATA_DIR"))

    known_args, unknown_args = parser.parse_known_args()


    # Let's first of all access the arguments to see their respective values
    n_estimators = known_args.n_estimators
    random_state = known_args.random_state
    model_dir = known_args.model_dir
    train_data_path = known_args.training_data_path # This value is set to 'SM_CHANNEL_TRAIN' as an env var after the "mode.fit({'train': d1, 'test': d2})"
    test_data_path = known_args.testing_data_path # This value is set to 'SM_CHANNEL_TEST' as an env var after the "mode.fit({'train': d1, 'test': d2})"
    train_file = known_args.training_file
    test_file = known_args.testing_file

    print(f"Number of estimators: {n_estimators}")
    print(f"Random state: {random_state}")
    print(f"Model directory: {model_dir}")
    print(f"Training data path: {train_data_path}")
    print(f"Testing data path: {test_data_path}")
    print(f"Training file: {train_file}")
    print(f"Testing file: {test_file}")

    # print("SkLearn Version: ", sklearn.__version__)
    # print("Joblib Version: ", joblib.__version__)

    print('\n\n')
    print("[INFO] Reading data from the S3 channels")
    print()

    training_data_df: DF = pd.read_csv(os.path.join(known_args.training_data_path, known_args.training_file))
    testing_data_df: DF = pd.read_csv(os.path.join(known_args.testing_data_path, known_args.testing_file))
    
    features = list(training_data_df.columns)
    # label = features.pop(-1)

    print("Building training and testing datasets")
    print()

    # Train independent features dataframes
    X_train_df: DF = training_data_df[features]
    # Train dependent feature
    y_train = training_data_df['price_range']



    # Test independent features dataframes
    X_test_df: DF = testing_data_df[features]  
    # Test dependent feature
    y_test = testing_data_df['price_range']

    # print('Column order: ')
    # print(features)
    # print()
    
    # print("Label column is: ", label)
    # print()
    
    print("\nTraing and testing dataframes shapes: ")
    print()
    
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(f'X_train shape: \t {X_train_df.shape}\n')
    print(f'y_train shape: \t {y_train.shape}\n')
    print()
    
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(f'X_test shape: \t {X_test_df.shape}\n')
    print(f'y_test shape: \t {y_test.shape}\n')

    print('\n\n')
    
  
    print("Training RandomForest Model.....")
    print()

    # Define the parameter grid
    param_grid = {
        # 'n_estimators': [50, 100, 200], # Let me remove this hyper parameter since it will be passed as a command line argument. 
        # I could choose to pass the whole list [50, 100, 200] as a command line argument by using the argparse's 'nargs' 
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10]
    }
    

    # Let's first of all initialize the model
    model =  RandomForestClassifier(n_estimators=known_args.n_estimators, random_state=known_args.random_state, verbose = 3,n_jobs=-1)
    
    
    # And then, initialize Grid Search
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)


    # And then let's fit Grid Search
    # model.fit(X_train, y_train)
    grid_search.fit(X_train_df.values, y_train.values)
    
    print(f'\nGrid Search best parameters: \n {grid_search.best_estimator_}')



    # Let's Get the best model
    best_model = grid_search.best_estimator_

    print()
    
    # Let's specify where we are going to dump our model
    # model_path = os.path.join(known_args.model_dir, "model.joblib")
    # joblib.dump(model,model_path)


    # Let's now, save the best model
    model_path = os.path.join(known_args.model_dir, "best_model.joblib")
    joblib.dump(best_model, model_path)

    
    print(f'The best model is  persisted at:\t {model_path}')


    print()

    
    test_y_pred = best_model.predict(X_test_df)
    test_accuracy = accuracy_score(y_test,test_y_pred)


    test_classification_report = classification_report(y_test,test_y_pred)

    print('\n\n')


    print("---- METRICS RESULTS FOR TESTING DATA ----")

    print()
    
    print(f'Total number of testing data points is \t: {X_test_df.shape[0]}\n\n')

    print(f'Testing accuracy is \t: {test_accuracy}')

    print(f'Testing classification report is \n: {test_classification_report}')
    
    print('\n\n')
    
    print('END !!!')

Overwriting train.py


In [63]:
# By default, our model will be stored in /opt/ml/

### _`Now, before sending this training script to sagemaker container and create and endpoint, let's test it here in our notebook first `_

In [64]:
# --training-file = train.csv by default
# --testing-file = test.csv by default

_`We can even refuse to give the hyper parameter values, it will still work, since it will take the default values if no value is given`_

In [65]:
! python train.py  --model-dir ./ \
                   --training-data-path ./ \
                   --testing-data-path ./ 

[INFO] Extracting the passed arguments from the command line


Number of estimators: 100
Random state: 0
Model directory: ./
Training data path: ./
Testing data path: ./
Training file: train.csv
Testing file: test.csv



[INFO] Reading data from the S3 channels

Building training and testing datasets


Traing and testing dataframes shapes: 

---- SHAPE OF TRAINING DATA (85%) ----
X_train shape: 	 (1700, 21)

y_train shape: 	 (1700,)


---- SHAPE OF TESTING DATA (15%) ----
X_test shape: 	 (300, 21)

y_test shape: 	 (300,)




Training RandomForest Model.....

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
building tree 1 of 100
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
building tree 3 of 100
building tree 2 of 100
building tree 1 of 100
building tree 4 of 100
building tree 2 of 100
building tree 4 of 100
building tree 5 of 100
building tree 3 of 100
build

<br><br><br><br><br><br><br><br><br>

In [66]:
# ! python train.py --n_estimators 100 \
#                    --random_state 0 \
#                    --model-dir ./ \
#                    --training-data-path ./ \
#                    --training-data-path ./ 

## _`7. Train script inside Sagemaker container. `_

#### _`Now, we need to give this training script to sagemaker sklearn container, as a training job in order for it to train the model, dump it to S3, and create an endpoint for inference`_

In [ ]:
from sagemaker.sklearn.estimator import SKLearn

# Let's first specify the machine learning framework we want to use (Skearn) version for  reproducibility, 
# as different versions of a framework might have different features, bug fixes, or performance
FRAMEWORK_VERSION = "0.23-1" 

sklearn_estimator = SKLearn(
    entry_point="train.py", # This is the file that will be executed 
    role=get_execution_role(), # The sagemaker will need execution role get be able to access S3 buckets (for the train and validation channels)
    instance_count=1, # This is the number of EC2 instance we woul;d like to lauch
    instance_type="ml.m5.large", # This is the type of the EC2 instance
    framework_version=FRAMEWORK_VERSION,
    base_job_name="mobile-price-prediction-model-training-job", # This id the name we would like to give to our training job in the sagemaker
    
    # Any hyper parameter that is provided in a dictionary format, sagemaker will pass all these key-values as a 
    # command line arguments to the training script
    # We can send an empty dictionary and the argparser default values will be used
    
    hyperparameters={ # This are the hyper parameters that we will provide in a dictionary format
        #"n_estimators": 100, # If not provided, the default value of the parser.add_argument will be used
        #"random_state": 0,  # If not provided, the default value of the parser.add_argument will be used
    },
    use_spot_instances = True, # We will use spot instances to save up to 92% of the norrmal cost
    
    max_wait = 7200,
    max_run = 3600
    )


'\n    max_wait = 7200\n    This parameter specifies the maximum amount of time (in seconds) that SageMaker will wait for Spot Instances to become available. \n    In this case, 7200 seconds equals 2 hours. If Spot Instances are not available within this time frame, the job will not start.\n\n    Here are the possible scenarios:\n\n    If Spot Instances become available within 7200 seconds: The job will start using the Spot Instances.\n    If Spot Instances are not available within 7200 seconds: The job will not start with Spot Instances. \n    Depending on your configuration, SageMaker might fall back to using On-Demand Instances if you have allowed for that option. \n    If not, the job will not proceed.\n\n    So, the max_wait parameter is essentially a timeout for waiting for Spot Instances to become available. \n    If the timeout is reached without Spot Instances being available, \n    the job’s behavior will depend on whether you have configured a fallback to On-Demand Instances

#### _`Let's now launch training job, with asynchronous call`_

_`In this context, an asynchronous call refers to starting the training job without waiting for it to complete before moving on to the next line of code. This allows your program to continue executing other tasks while the training job runs in the background.`_

In [68]:
training_data_path

's3://sagemaker-diangana-bucket/data/train/train.csv'

In [69]:
testing_data_path

's3://sagemaker-diangana-bucket/data/test/test.csv'

In [70]:
# launching training job, with asynchronous call
sklearn_estimator.fit({"train": training_data_path, "test": testing_data_path}, wait=True)
# sklearn_estimator.fit({"train": datapath}, wait=True)

INFO:sagemaker:Creating training-job with name: mobile-price-prediction-model-training--2024-11-05-12-50-33-283


2024-11-05 12:50:33 Starting - Starting the training job...
2024-11-05 12:50:54 Starting - Preparing the instances for training...
2024-11-05 12:51:18 Downloading - Downloading input data...
2024-11-05 12:51:44 Downloading - Downloading the training image...
2024-11-05 12:52:25 Training - Training image download completed. Training in progress.2024-11-05 12:52:29,033 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-11-05 12:52:29,036 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-05 12:52:29,071 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-11-05 12:52:29,225 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-05 12:52:29,236 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-05 12:52:29,248 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-05 12:5

## _`8. Store Model Artifacts(model.tar.gz) into the S3 Bucket. `_

In [ ]:
sklearn_estimator.latest_training_job.wait(logs="None") # This line waits for the latest training job to complete.

artifact = boto3_sagemaker_client.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"] #  Retrieves detailed information about the lastest training job. This line extracts the S3 URI where the trained model artifacts are stored. 

print(f"\n\nModel artifact persisted at: {artifact} \n\n")

# print(f'The detailed infromation about the latest training job is: \n{boto3_sagemaker_client.describe_training_job(
#     TrainingJobName=sklearn_estimator.latest_training_job.name
# )}')


2024-11-05 12:52:58 Starting - Preparing the instances for training
2024-11-05 12:52:58 Downloading - Downloading the training image
2024-11-05 12:52:58 Training - Training image download completed. Training in progress.
2024-11-05 12:52:58 Uploading - Uploading generated training model
2024-11-05 12:52:58 Completed - Training job completed


Model artifact persisted at: s3://sagemaker-eu-west-3-050752617294/mobile-price-prediction-model-training--2024-11-05-12-50-33-283/output/model.tar.gz 




'\nsklearn_estimator.latest_training_job.wait(logs="None")\nsklearn_estimator.latest_training_job.wait(logs="None"): This line waits for the latest training job to complete. \nThe logs="None" parameter indicates that you do not want to display the logs while waiting. \nThis can be useful if you want to avoid cluttering your output with log messages.\n\n\n\nartifact = sm_boto3.describe_training_job(...): This line uses the SageMaker Boto3 client (sm_boto3) to describe the training job. \nThe describe_training_job method retrieves detailed information about the training job.\n\nTrainingJobName=sklearn_estimator.latest_training_job.name: This parameter specifies the name of the training job you want to describe. \nIt uses the name of the latest training job from the sklearn_estimator.\n\n["ModelArtifacts"]["S3ModelArtifacts"]: This part of the line extracts the S3 URI where the trained model artifacts are stored. \nThe ModelArtifacts key contains information about the model artifacts, and

#### _`Now that we know the location (path) to the model.tar.gz artifacts, we can deploy a sagemaker endpoint for our trained model and then test it using API calls`_

## _`9. Deploy Sagemaker Endpoint(API) for trained model, and test it. `_

In [74]:
%%writefile inference.py

import joblib
import os

"""
In my case, I don't require to have predict_fn(). 
This predict_fn is used to change the prediction. Eg: Adding type or adding more layers to my prediction
"""




def input_fn(request_body, request_content_type):
    print(request_body)
    print(request_content_type)
    if request_content_type == "text/csv":  # If the incomming request body data is a 'text/csv' type
        request_body = request_body.strip() # We will make sure that there is no leading or ending space character in the request body and making sure that the request data is clean 
                                            # and ready for being transformed into a pandas dataframe 
        try:
            df = pd.read_csv(StringIO(request_body), header=None) # Since the the incomming request will come as a stream data
            return df
        
        except Exception as e:
            print(e)
    else:
        return """Please use Content-Type = 'text/csv' and, send the request!!""" 




# The model_fn function is required
def model_fn(model_dir):
    model_path = os.path.join(model_dir, "model.joblib")
    model = joblib.load(model_path)
    return model



def predict_fn(input_data, model):
    if type(input_data) != str: # Because we have transformed the received request body data into a pandas dataframe, so we will only predict if the request body data is a dataframe and not a string
        prediction = model.predict(input_data)
        print(prediction)
        return prediction
    else:
        return input_data
        
        

def output_fn(prediction, content_type):
    import json
    if content_type == 'text/csv': # In our case, we want the response from the model to be a list of numbers with the lenght of the incoming request data points lenght(text/csv) and not a 'application/json':
        response_body = json.dumps(prediction.tolist())
        return response_body
    else:
        raise ValueError("Unsupported content type: {}".format(content_type))


Writing inference.py


In [78]:
artifact

's3://sagemaker-eu-west-3-050752617294/mobile-price-prediction-model-training--2024-11-05-12-50-33-283/output/model.tar.gz'

### _`Let's now register the model`_

In [75]:
# Let's first read the trained tar.gz file from S3 and make it available in SageMaker as a SKlearn classic model. 
# So that we will be able to create an endpoint for it
# And for that that we will import 'SKLearnModel' from sagemaker.sklearn.model

from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name: str = "mobile-price-prediction-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())


model = SKLearnModel(
    name=model_name,
    model_data=artifact,
    role=get_execution_role(),
    entry_point="inference.py", # This is my inference logic script that contains the model_fn, input_fn, predict_fn, output_fn function
    framework_version=FRAMEWORK_VERSION,
)

In [ ]:
# After making the Sklearn model available in sagemaker, let's now deploy the model and get an endpoint of it

endpoint_name = "mobile-price-prediction-endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

print(f'\n\nEndpointName={endpoint_name}\n\n')

predictor = model.deploy(

    initial_instance_count=1, # This is the initoial number of instances we would like to deploy our endpoint onto. We can use later ASG (Auto Scaling Groups) 
                            # To scal up or scale down along with load balancing

    instance_type='ml.m5.large',  # "ml.m4.xlarge" instance type ml.m4.xlarge is not supported for the chosen region eu-west-3
    
    endpoint_name=endpoint_name,
)

'\ninitial_instance_count=1, \nThe number of instances to launch initially.\n\nWhen I say “launch initially” in the context of initial_instance_count, it refers to the number of EC2 instances that will be started \nwhen you first deploy your model to an endpoint.\n\nHere’s a bit more detail:\n\nInitial Deployment: When you deploy your model, SageMaker will create an endpoint and launch the specified number of instances \nto host your model. For example, if you set initial_instance_count=1, SageMaker will start one instance to serve your model.\n\nScalability: This parameter sets the baseline for your deployment. You can later adjust the number of instances based on the traffic \nand performance requirements. SageMaker also supports automatic scaling, where the number of instances can increase or decrease based \non the load.\n'

## _`let's now test the endpoint`_

In [ ]:
# test_data_df.drop('price_range', axis=1)[0:2].values.tolist()

[[1454.0,
  1.0,
  0.5,
  1.0,
  1.0,
  0.0,
  34.0,
  0.7,
  83.0,
  4.0,
  3.0,
  250.0,
  1033.0,
  3419.0,
  7.0,
  5.0,
  5.0,
  1.0,
  1.0,
  0.0],
 [1092.0,
  1.0,
  0.5,
  1.0,
  10.0,
  0.0,
  11.0,
  0.5,
  167.0,
  3.0,
  14.0,
  468.0,
  571.0,
  737.0,
  14.0,
  4.0,
  11.0,
  0.0,
  1.0,
  0.0]]

In [ ]:
test_data_df.drop('price_range', axis=1).head(3).values

In [ ]:
print(predictor.predict(test_data_df.drop('price_range', axis=1).head(3).values))

[3 0]


_`We can now use this endpoint in Postman, or in python application or as a REST API (ASW API Gateway)`_

## _`Important!!! Let's not forget to delete the endpoint !`_

In [84]:
boto3_sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '6ff8f497-b7f4-4cfc-a082-881655ce2b24',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6ff8f497-b7f4-4cfc-a082-881655ce2b24',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 02 Nov 2024 22:27:36 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}